## チェックポイントでモデル構造の復元を試す
- Reference
  - https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja
  - https://www.tensorflow.org/guide/checkpoint

In [1]:
import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.2.0'

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

print(train_images.shape)

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

print(train_images.shape)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(1000, 784)


### モデルを作成して学習させつつ CheckpointManager で保存する

In [3]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = 'data/checkpoint_architecture/cp.ckpt'

ckpt = tf.train.Checkpoint(model_1=model)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [5]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs = 10, 
          validation_data = (test_images, test_labels),
          verbose=0)

In [6]:
print(ckpt_manager.latest_checkpoint)

None


In [7]:
ckpt_manager.save()

print(ckpt_manager.latest_checkpoint)

data/checkpoint_architecture/cp.ckpt/ckpt-1


### ダミーのモデルを作成して CheckpointManager から復元する

In [8]:
def create_model_dummy():
  model = tf.keras.models.Sequential()

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


model2 = create_model_dummy()

In [9]:
ckpt2 = tf.train.Checkpoint(model_1=model2)

ckpt_manager2 = tf.train.CheckpointManager(ckpt2, checkpoint_path, max_to_keep=5)

In [10]:
print(ckpt_manager2.latest_checkpoint)

data/checkpoint_architecture/cp.ckpt/ckpt-1


In [11]:
ckpt2.restore(ckpt_manager2.latest_checkpoint)

In [12]:
model2.summary()

ValueError: ignored

### モデル作成して復元する

In [13]:
model3 = create_model()
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [14]:
ckpt3 = tf.train.Checkpoint(model_1=model3)

ckpt_manager3 = tf.train.CheckpointManager(ckpt3, checkpoint_path, max_to_keep=5)

In [15]:
print(ckpt_manager3.latest_checkpoint)

data/checkpoint_architecture/cp.ckpt/ckpt-1


In [16]:
ckpt3.restore(ckpt_manager3.latest_checkpoint)

In [17]:
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
